# Beam Search Pytorch

Beam search作为最基础的搜索解码算法，我们关注其实现细节。

给定束beam_size=2情况，那么在t=1时刻，会解码出beam_size=2条路径

在t=2时刻，2条路径各自有beam_size个候选token，那么就会产生2x2条路径，那么我们可以依照路径的概率和来选择beam条路径

![](./image/beam-search.png)

In [2]:
import torch
import torch.nn.functional as F
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name, clean_up_tokenization_spaces=True)

text = "Once upon a time"
input_ids = tokenizer.encode(text, return_tensors="pt")
print(input_ids)

tensor([[7454, 2402,  257,  640]])

In [3]:
max_length = 4
beam_width = 3

# 初始化beam
beams = [(input_ids, 0.0)]
print(beams)
completed_beams = []

for _ in range(max_length):
    new_beams = []

    for beam_input_ids, beam_score in beams:
        # 如果遇到了结束标记，将该beam加入到已完成列表中
        if beam_input_ids[0, -1].item() == tokenizer.eos_token_id:
            completed_beams.append((beam_input_ids, beam_score))
            continue

        # 不直接预测next token
        # 而是拿出logits里找到beam个候选的next token
        with torch.no_grad():
            outputs = model(beam_input_ids)
            next_token_logits = outputs.logits[:, -1, :]
        next_token_scores = F.log_softmax(next_token_logits, dim=-1)
        top_k_scores, top_k_tokens = torch.topk(next_token_scores, 
                                                beam_width, dim=-1)

        # 在每个候选的token里都统计路径分数
        for i in range(beam_width):
            next_token = top_k_tokens[0, i].unsqueeze(0).unsqueeze(0)
            next_score = top_k_scores[0, i].item()
            new_input_ids = torch.cat([beam_input_ids, next_token], dim=-1)
            new_score = beam_score + next_score
            new_beams.append((new_input_ids, new_score))

    # 注意到beam search候选路径会有 beam_size^2条，请思考为什么？
    print(len(new_beams))

    # 选择得分最高的beam_width个beam
    beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[:beam_width]
    print(len(beams))
    print(beams)

    # 如果所有beam都已完成，则停止生成
    if len(completed_beams) == beam_width:
        break
        
# 如果还有未完成的beam，将它们加入到已完成列表中。
# beam_size*2 >= 实际路径 >= beam_size
# 有额外的beam产生的原因是？
# 在beam search解码过程中，有路径遇到eos token终止搜索，但是beam_size仍不会变化
completed_beams.extend(beams)
print(len(completed_beams))

[(tensor([[7454, 2402,  257,  640]]), 0.0)]

3

3

[
    (tensor([[7454, 2402,  257,  640,   11]]), -0.8511835932731628),
    (tensor([[7454, 2402,  257,  640,  262]]), -2.739619016647339),
    (tensor([[7454, 2402,  257,  640,  314]]), -3.2028720378875732)
]

9

...

[
    (tensor([[7454, 2402,  257,  640,   11,  340,  373]]), -5.191973507404327),
    (tensor([[7454, 2402,  257,  640,   11,  314,  373]]), -5.565997660160065),
    (tensor([[7454, 2402,  257,  640,   11,  314,  550]]), -6.277988135814667)
]

9

3

[
    (tensor([[7454, 2402,  257,  640,   11,  340,  373,  257]]), -8.047162353992462),
    (tensor([[7454, 2402,  257,  640,   11,  340,  373,  262]]), -8.205495178699493),
    (tensor([[7454, 2402,  257,  640,   11,  340,  373,  531]]), -8.311688721179962)
]

3

In [4]:
for beams in completed_beams:
    generated_text = tokenizer.decode(beams[0][0], skip_special_tokens=True)
    print(generated_text)

Once upon a time, it was a

Once upon a time, it was the

Once upon a time, it was said

# beam searching路径选择

In [5]:
x = [
    ["a", 0.04],
    ["b", 0.7],
    ["d", 0.2],
    ["e", 0.06],
]
result = sorted(x, key=lambda x: x[1], reverse=True)[:2]
print(result)

[['b', 0.7], ['d', 0.2]]

# 补充问题思考

Beam Search相较greedy的时间复杂度是多少？

Beam Search是全局最优的解码路径吗？

Beam Search的候选路径有多少条？

Beam Search里如果有路径提前遇见EOS后，后续的解码路径的beam是否会改变？